In [1]:
file_path = '/Users/lykofos/Desktop/study/SEM-3/CS786/A4/rational.py'


with open(file_path, 'r') as file:
    
    rational_model_code = file.read()

rational_model_code[:1000]  


'\n# Implementation of Anderson\'s venerable "rational" model of categorization.\n# Assumes that stimuli were generated by a mixture of Gaussian distributions;\n# rather than compute the full Bayesian posterior, it views items sequentially\n# and assigns each to the maximum a posteriori cluster.\n#\n# At the end it is presented with a stimulus with one item missing, and\n# predicts the probability that its value is a \'0\' or a \'1\'.\n#\n# Implemented in python by John McDonnell\n#\n# References: Anderson (1990) and Anderson (1991),\n\nimport numpy as np\nfrom random import shuffle\n\n#Utility functions:\n\nclass dLocalMAP:\n    """\n    See Anderson (1990, 1991)\n    \'Categories\' renamed \'clusters\' to avoid confusion.\n    Discrete version.\n    \n    Stimulus format is a list of integers from 0 to n-1 where n is the number\n    of possible features (e.g. [1,0,1])\n    \n    args: c, alphas\n    """\n    \n    def __init__(self, args):\n        self.partition = [[]]\n        self

In [2]:
import pandas as pd

# Load the training and test datasets
train_data_path = '/Users/lykofos/Desktop/study/SEM-3/CS786/A4/X.csv'
test_data_path = '/Users/lykofos/Desktop/study/SEM-3/CS786/A4/y.csv'

# Reading the datasets
train_data = pd.read_csv(train_data_path, header=None)
test_data = pd.read_csv(test_data_path, header=None)

train_data.head(), test_data.head()


(    0   1  2
 0  48  58  1
 1  54  62  1
 2  48  56  1
 3  46  62  1
 4  47  59  1,
     0   1
 0  74  67
 1  69  63
 2  92  81
 3  64  61
 4  66  84)

In [3]:
import numpy as np


X_train = train_data[[0, 1]].values  
y_train = train_data[2].values       


X_test = test_data.values

# Define a modified Rational Model function for prediction based on Gaussian assumptions
class RationalModel:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.categories = {}

    def fit(self, X, y):
        """Fit the model using the training data."""
        for label in np.unique(y):
            subset = X[y == label]
            mean = np.mean(subset, axis=0)
            cov = np.cov(subset, rowvar=False)
            self.categories[label] = (mean, cov + np.eye(cov.shape[0]) * 1e-6)  # Regularize cov matrix

    def predict(self, X):
        """Predict categories for test data."""
        predictions = []
        for x in X:
            posteriors = []
            for label, (mean, cov) in self.categories.items():
                diff = x - mean
                likelihood = np.exp(-0.5 * diff.T @ np.linalg.inv(cov) @ diff)
                posteriors.append((label, likelihood))
            predictions.append(max(posteriors, key=lambda item: item[1])[0])
        return np.array(predictions)

# Train the model
model = RationalModel()
model.fit(X_train, y_train)

# Predict on the test dataset
y_pred = model.predict(X_test)
y_pred


array([3, 2, 3, 2, 2, 3, 2, 2, 2, 1])